In [1]:
video_ids = range(1, 6)

In [2]:
# Load the JSON metadata for each video and store the paths, hits, and serves
import json

data = {}
for id in video_ids:
    with open(f'data/json/video{id}.json') as f:
        video_data = json.load(f)

        video_start = video_data['match']['start']
        video_end = video_data['match']['end']
        
        # Create a dictionary to store the labels for each frame
        video_labels = {
            frame: {
                'shot': None,
                'player': None,
                'type': None,
                'side': None
            }
            for frame in range(video_start, video_end)
        }

        # Add the labels for each hit
        for hit in video_data['hits']:
            hit_start = int(hit['start'])
            hit_end = int(hit['end'])
            hit_label = hit['custom']

            for frame in range(hit_start, hit_end):
                video_labels[frame] = {
                    'shot': 'Hit',
                    'player': hit_label['Player'],
                    'type': hit_label['Type'],
                    'side': hit_label['Side']
                }

        # Add the labels for each serve
        for serve in video_data['serves']:
            serve_start = int(serve['start'])
            serve_end = int(serve['end'])
            serve_label = serve['custom']

            for frame in range(serve_start, serve_end):
                video_labels[frame] = {
                    'shot': 'Serve',
                    'player': serve_label['Player'],
                    'type': serve_label['Result'],
                    'side': None
                }

        data[id] = video_labels

In [3]:
import numpy as np

# Convert the video labels into a numpy array
num_frames = sum([len(video_labels) for video_labels in data.values()])
labels = np.zeros((num_frames, 4), dtype=object)

frame = 0
for video_id, video_labels in data.items():
    for frame, labels in video_labels.items():
        labels[frame] = [labels['shot'], labels['player'], labels['type'], labels['side']]
        frame += 1